# My exercise of MLP for MNIST

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
# hyper parameters
n_input = 28 ** 2
n_layer1 = 256
n_layer2 = 256
n_label = 10

learning_rate = 0.001
batch_size = 128
n_epoch = 15
display_per_epoch = 1

In [22]:
# define MLP
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_layer1])),
    'h2': tf.Variable(tf.random_normal([n_layer1, n_layer2])),
    'out': tf.Variable(tf.random_normal([n_layer2, n_label]))
}

biases = {
    'h1': tf.Variable(tf.random_normal([n_layer1])),
    'h2': tf.Variable(tf.random_normal([n_layer2])),
    'out': tf.Variable(tf.random_normal([n_label]))
}

x = tf.placeholder('float', [None, n_input])
y = tf.placeholder('float', [None, n_label])

layer1 = tf.matmul(x, weights['h1']) + biases['h1']
layer1 = tf.nn.relu(layer1)
layer2 = tf.matmul(layer1, weights['h2']) + biases['h2']
layer2 = tf.nn.relu(layer2)
pred = tf.matmul(layer2, weights['out']) + biases['out']

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [24]:
# train
import numpy as np
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(n_epoch):
        costs = []
        # train each mini-batch
        for mini_batch in range(int(mnist.train.num_examples/batch_size)+1):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            _, batch_cost = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
            costs.append(batch_cost)
        if epoch % display_per_epoch == 0:
            print('Epoch=%04d, cost = %.9f' % (epoch+1, np.mean(costs)))
    print('training finished')
    
    # test accuracy
    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, 'float'))
    print('Accuracy@test: %.9f' % accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))

Epoch=0001, cost = 165.694061279
Epoch=0002, cost = 43.874183655
Epoch=0003, cost = 28.677154541
Epoch=0004, cost = 20.203235626
Epoch=0005, cost = 15.119401932
Epoch=0006, cost = 11.513390541
Epoch=0007, cost = 8.851198196
Epoch=0008, cost = 6.844130993
Epoch=0009, cost = 5.283704281
Epoch=0010, cost = 4.062216282
Epoch=0011, cost = 3.204838991
Epoch=0012, cost = 2.344624281
Epoch=0013, cost = 1.827739716
Epoch=0014, cost = 1.396134615
Epoch=0015, cost = 1.084728479
training finished
Accuracy@test: 0.940500021


# use tflearn below

In [12]:
import tflearn
import tflearn.datasets.mnist as mnist_tflearn

trainX, trainY, testX, testY = mnist_tflearn.load_data('MNIST_data',one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [19]:
tf.reset_default_graph()
net = tflearn.input_data([None, n_input])
net = tflearn.fully_connected(net, n_layer1, activation='ReLU')
net = tflearn.fully_connected(net, n_layer2, activation='ReLU')
net = tflearn.fully_connected(net, n_label, activation='softmax')
net = tflearn.regression(net,
                         loss='categorical_crossentropy',
                         optimizer='adam',
                         learning_rate=learning_rate)
model = tflearn.DNN(net)

model.fit(trainX, trainY,
         validation_set = 0.1, batch_size=batch_size, n_epoch = n_epoch,
         show_metric=True)

Training Step: 5804  | total loss: 0.33323 | time: 4.726s
| Adam | epoch: 015 | loss: 0.33323 - acc: 0.9734 -- iter: 49408/49500
Training Step: 5805  | total loss: 0.30500 | time: 5.744s
| Adam | epoch: 015 | loss: 0.30500 - acc: 0.9760 | val_loss: 0.11207 - val_acc: 0.9700 -- iter: 49500/49500
--


In [20]:
pred = np.argmax(model.predict(testX), axis=1)
actual = testY.argmax(axis=1)
test_accuracy = np.mean(pred==actual,  axis=0)
print('test_accuracy: %.6f' % test_accuracy)

test_accuracy: 0.974200
